In [1]:
import numpy as np
import pandas as pd

загружаем данные

In [9]:
cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

In [3]:
x_test_prediction = np.loadtxt('x_test_prediction.csv',delimiter=',')
x_test_real = np.loadtxt('x_test_real.csv', delimiter=',')

функции

In [7]:
def BinimialPrediction(X, treshold = 0.5):
    result = np.zeros(X.shape)
    
    if type(treshold) == int or type(treshold) == float:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                if X[i, j] >= treshold:
                    result[i, j] = 1
    elif type(treshold) == list:
        for i in range(X.shape[0]):
            for j in range(X.shape[1]):
                if X[i, j] >= treshold[j]:
                    result[i, j] = 1
    else:
        print('treshold type must be int, float or list')
    
    return(result)

def FBettaScore(x_true, x_predicted, betta = 2):
    if len(x_true) == len(x_predicted):
        tp = 0
        fp = 0
        fn = 0

        for i in range(len(x_predicted)):
            if x_true[i] == 1 and x_predicted[i] == 1:
                tp += 1
            
            if x_true[i] == 0 and x_predicted[i] == 1:
                fp += 1
            
            if x_true[i] == 1 and x_predicted[i] == 0:
                fn += 1
        
        if tp == 0 or (tp + fp) == 0 or (tp + fn) == 0:
            return(0)
        else:
            precision = tp/(tp + fp)
            recall = tp/(tp + fn)
            
            return((1 + betta**2)*precision*recall/(betta**2*precision + recall))
    else:
        print('FBettaScore error! len(x_true) != len(x_predicted)')

def AvgFBettaScore(x_true, x_predicted, betta = 2, treshold = 0.5):
    result = 0
    n = x_true.shape[0]
    
    x_predicted = BinimialPrediction(x_predicted, treshold)
    
    for i in range(n):
        result += FBettaScore(x_true[i, :], x_predicted[i, :], betta)
    
    return(result/n)


In [10]:
x_test_binomial = BinimialPrediction(x_test_prediction, treshold = 0.2)
AvgFBettaScore(x_test_real, x_test_prediction, betta = 2, treshold = 0.2)

0.9043313365016242

In [21]:
errors = np.zeros((len(cathegories), 4))

for i in range(x_test_binomial.shape[0]):
    for j in range(x_test_binomial.shape[1]):
        predicted = x_test_binomial[i, j]
        real = x_test_real[i, j]
        
        
        if predicted == 1 and real == 1:
            errors[j, 0] += 1
        
        if predicted == 1 and real == 0:
            errors[j, 1] += 1
        
        if predicted == 0 and real == 1:
            errors[j, 2] += 1
        
        if real == 1:
            errors[j, 3] += 1

errors = errors.astype(int)
errors_df = pd.DataFrame(errors, index= cathegories, columns= ['tp', 'fp', 'fn', 'cnt'])
errors_df

,tp,fp,fn,cnt
agriculture,13783,5456,1021,14804
artisinal_mine,262,126,129,391
bare_ground,298,542,736,1034
blooming,20,54,368,388
blow_down,11,18,91,102
clear,33784,2485,510,34294
cloudy,2303,965,177,2480
conventional_mine,22,20,92,114
cultivation,3598,4783,1754,5352
habitation,2978,3219,1543,4521


In [31]:
x_test_binomial_weather = x_test_binomial[:, [cathegories.index(i) for i in ['clear', 'cloudy', 'haze', 'partly_cloudy']]]

In [33]:
cnt = 0
s = 0

for i in range(x_test_binomial_weather.shape[0]):
    if sum(x_test_binomial_weather[i, :]) > 1:
        cnt += 1
        s += sum(x_test_binomial_weather[i, :])

cnt, cnt/x_test_binomial_weather.shape[0], s/cnt

(3971, 0.08174987133299022, 2.0418030722739866)